## Demo

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
import pandas as pd

bikes = pd.read_csv('./datasets/austin_bikes_final.csv')
bikes

,bikeid,start_time,duration_minutes,start_station_name,end_station_name,start_latitude,start_longitude,end_latitude,end_longitude
0,8,2015-03-19 19:12:00,41,Waller & 6th St.,Trinity & 6th Street,30.26461,-97.73049,30.26735,-97.73933
1,468,2016-03-21 21:17:48,7,Waller & 6th St.,Trinity & 6th Street,30.26461,-97.73049,30.26735,-97.73933
2,107,2016-05-29 13:09:31,35,Waller & 6th St.,Trinity & 6th Street,30.26461,-97.73049,30.26735,-97.73933
3,428,2016-02-20 14:40:49,12,Waller & 6th St.,Trinity & 6th Street,30.26461,-97.73049,30.26735,-97.73933
4,572,2015-07-09 16:12:00,4,Waller & 6th St.,Trinity & 6th Street,30.26461,-97.73049,30.26735,-97.73933
...,...,...,...,...,...,...,...,...,...
614423,183,2017-07-11 18:56:51,43,Congress & Cesar Chavez,Congress & Cesar Chavez,30.26332,-97.74508,30.26332,-97.74508
614424,849,2017-07-13 06:02:13,5,Congress & Cesar Chavez,Boardwalk West,30.26332,-97.74508,30.25457,-97.74258
614425,209,2017-07-31 11:20:23,31,Congress & Cesar Chavez,Boardwalk West,30.26332,-97.74508,30.25457,-97.74258
614426,2048,2017-07-21 06:08:30,5,Congress & Cesar Chavez,Boardwalk West,30.26332,-97.74508,30.25457,-97.74258


In [7]:
from jaccard_workload_approximator import JaccardWorkloadApproximator

workload_approximator = JaccardWorkloadApproximator()
workload = workload_approximator.get_workload_approximation(30)

============ starting iteration 1   ============
Created query:
SELECT _id FROM public.austin_bikes_1000 WHERE duration_minutes BETWEEN 477.9264641429193 AND 504.2417740640721 AND start_latitude BETWEEN 30.249567428173478 AND 30.272865299773947
iteration took: 0.00 ms
============ starting iteration 2   ============
Created query:
SELECT _id FROM public.austin_bikes_1000 WHERE end_longitude BETWEEN -97.7483069147178 AND -97.71174853486328 AND start_station_name IN ('State Capitol Visitors Garage @ San Jacinto & 12th' , 'ACC - West & 12th Street' , 'State Capitol @ 14th & Colorado' , 'Guadalupe & 6th' , 'Bullock Museum @ Congress & MLK' , '5th & San Marcos' , '2nd & Congress' , 'Barton Springs & Riverside' , 'MoPac Pedestrian Bridge @ Veterans Drive' , '6th & Congress' , '4th & Congress' , 'Riverside @ S. Lamar' , 'Nueces & 3rd' , 'Lavaca & 6th' , 'City Hall / Lavaca & 2nd' , '13th & San Antonio') AND bikeid BETWEEN 83.3154460111773 AND 1903.7693068474575
iteration took: 0.00 ms
=======

In [8]:
from saqp_manager import SaqpManager

saqp_manager = SaqpManager([query[0] for query in workload], [query[1] for query in workload], 30)
sample = saqp_manager.get_sample()
sample

Start running LazyGreedy with type UC
============ starting iteration 1   ============
iteration took: 95.04 ms
============ starting iteration 2   ============
iteration took: 344.72 ms
============ starting iteration 3   ============
iteration took: 403.19 ms
============ starting iteration 4   ============
iteration took: 405.10 ms
============ starting iteration 5   ============
iteration took: 487.55 ms
============ starting iteration 6   ============
iteration took: 522.08 ms
============ starting iteration 7   ============
iteration took: 570.15 ms
============ starting iteration 8   ============
iteration took: 612.88 ms
============ starting iteration 9   ============
iteration took: 663.93 ms
============ starting iteration 10   ============
iteration took: 699.29 ms
============ starting iteration 11   ============
iteration took: 691.14 ms
============ starting iteration 12   ============
iteration took: 724.34 ms
============ starting iteration 13   ============
iteration 

[{'_id': 614429,
  'bikeid': 693,
  'start_time': datetime.datetime(2016, 3, 20, 19, 46, 36, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=120, name=None)),
  'duration_minutes': 8,
  'start_station_name': '5th & Bowie',
  'end_station_name': '8th & Congress',
  'start_latitude': 30.2696,
  'start_longitude': -97.75332,
  'end_latitude': 30.2698,
  'end_longitude': -97.74186},
 {'_id': 614430,
  'bikeid': 577,
  'start_time': datetime.datetime(2016, 3, 1, 16, 7, 50, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=120, name=None)),
  'duration_minutes': 11,
  'start_station_name': 'Barton Springs @ Kinney Ave',
  'end_station_name': '5th & Bowie',
  'start_latitude': 30.262,
  'start_longitude': -97.76118,
  'end_latitude': 30.2696,
  'end_longitude': -97.75332},
 {'_id': 614431,
  'bikeid': 465,
  'start_time': datetime.datetime(2016, 5, 5, 7, 29, 25, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=180, name=None)),
  'duration_minutes': 13,
  'start_station_name': 'South Congress & Academ

In [9]:
df = pd.DataFrame(sample)
df

,_id,bikeid,start_time,duration_minutes,start_station_name,end_station_name,start_latitude,start_longitude,end_latitude,end_longitude
0,614429,693,2016-03-20 19:46:36+02:00,8,5th & Bowie,8th & Congress,30.26960,-97.75332,30.26980,-97.74186
1,614430,577,2016-03-01 16:07:50+02:00,11,Barton Springs @ Kinney Ave,5th & Bowie,30.26200,-97.76118,30.26960,-97.75332
2,614431,465,2016-05-05 07:29:25+03:00,13,South Congress & Academy,Capitol Station / Congress & 11th,30.25226,-97.74854,30.27260,-97.74127
3,614432,391,2014-06-28 09:12:00+03:00,10,State Capitol Visitors Garage @ San Jacinto & ...,2nd & Congress,30.27336,-97.73805,30.26408,-97.74355
4,614433,970,2017-05-16 17:41:07+03:00,10,6th & Congress,Davis at Rainey Street,30.26822,-97.74285,30.26019,-97.73845
5,614434,961,2014-03-28 18:12:00+03:00,8,Capitol Station / Congress & 11th,City Hall / Lavaca & 2nd,30.27260,-97.74127,30.26476,-97.74678
6,614435,775,2015-03-14 18:12:00+02:00,1,Barton Springs & Riverside,Barton Springs & Riverside,30.25870,-97.74872,30.25870,-97.74872
7,614436,668,2015-03-29 09:12:00+03:00,28,Palmer Auditorium,5th & Bowie,30.25971,-97.75346,30.26960,-97.75332
8,614437,8,2014-10-27 16:12:00+02:00,3,San Jacinto & 8th Street,Convention Center / 4th St. @ MetroRail,30.26912,-97.73986,30.26483,-97.73900
9,614438,37,2015-10-04 17:12:16+03:00,6,Barton Springs @ Kinney Ave,Zilker Park,30.26200,-97.76118,30.26590,-97.76822


In [55]:
query = input('enter query please...').replace("_id", "*")
query

"SELECT * FROM public.austin_bikes_1000 WHERE start_station_name IN ('Palmer Auditorium')"

In [56]:
from data_access import DataAccess

dataAccess = DataAccess()
# query = "SELECT _id FROM public.austin_bikes_1000 WHERE start_station_name IN ('Palmer Auditorium')".replace("_id", "*")
truth_query_df = dataAccess.select_to_df(query)
truth_query_df

,_id,bikeid,start_time,duration_minutes,start_station_name,end_station_name,start_latitude,start_longitude,end_latitude,end_longitude
0,614436,668,2015-03-29 09:12:00+03:00,28,Palmer Auditorium,5th & Bowie,30.25971,-97.75346,30.26960,-97.75332
1,614442,551,2017-05-06 22:14:21+03:00,37,Palmer Auditorium,Plaza Saltillo,30.25971,-97.75346,30.26217,-97.72743
2,614494,185,2017-03-06 17:57:53+02:00,9,Palmer Auditorium,2nd & Congress,30.25971,-97.75346,30.26408,-97.74355
3,614602,432,2015-06-29 06:12:00+03:00,11,Palmer Auditorium,Convention Center / 4th St. @ MetroRail,30.25971,-97.75346,30.26483,-97.73900
4,614610,203,2016-01-10 11:12:44+02:00,25,Palmer Auditorium,MoPac Pedestrian Bridge @ Veterans Drive,30.25971,-97.75346,30.27466,-97.77028
5,614652,64,2016-03-15 21:20:38+02:00,28,Palmer Auditorium,Palmer Auditorium,30.25971,-97.75346,30.25971,-97.75346
6,614655,984,2015-11-30 13:12:56+02:00,7,Palmer Auditorium,5th & Bowie,30.25971,-97.75346,30.26960,-97.75332
7,614662,188,2017-04-19 13:45:30+03:00,68,Palmer Auditorium,Long Center @ South 1st & Riverside,30.25971,-97.75346,30.25941,-97.74971
8,614677,348,2015-03-14 16:12:00+02:00,80,Palmer Auditorium,Palmer Auditorium,30.25971,-97.75346,30.25971,-97.75346
9,614750,885,2017-03-08 07:06:53+02:00,5,Palmer Auditorium,Pfluger Bridge @ W 2nd Street,30.25971,-97.75346,30.26717,-97.75484


In [57]:
import pandasql as ps

sample_query_df = ps.sqldf(query.replace("public.austin_bikes_1000", "df"))
sample_query_df

,_id,bikeid,start_time,duration_minutes,start_station_name,end_station_name,start_latitude,start_longitude,end_latitude,end_longitude
0,614436,668,2015-03-29 09:12:00.000000,28,Palmer Auditorium,5th & Bowie,30.25971,-97.75346,30.26960,-97.75332
1,614442,551,2017-05-06 22:14:21.000000,37,Palmer Auditorium,Plaza Saltillo,30.25971,-97.75346,30.26217,-97.72743


In [53]:
len(sample) / dataAccess.select_one("SELECT count(*) as count from public.austin_bikes_1000")['count']

0.03

In [58]:
len(sample_query_df.index) / len(truth_query_df.index)

0.08